In [1]:
import keras

In [2]:
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='klNf6oE0FdirMnVg_5lZlC9luxa3pINITIX48MlCWJu5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'imageanalysis-donotdelete-pr-3zp79iioz5e4oo'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [2]:
pwd

'/home/wsuser/work'

In [5]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/Dataset/train_set')

In [ ]:
x_train=train_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/train_set',
                                          target_size=(128,128),batch_size=32,class_mode='binary')


In [6]:
x_test=test_datagen.flow_from_directory('/home/wsuser/work/Dataset/Dataset/test_set',
                                        target_size=(128,128),batch_size=32,class_mode='binary')


Found 121 images belonging to 2 classes.


In [7]:
#to define the linear Initialisation import sequential
from keras.models import Sequential
#to add layers import Dense
from keras.layers import Dense
#to create Convolutional kernel import convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer 
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [8]:
model=Sequential()

In [9]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layers
model.add(MaxPooling2D(pool_size=(2,2)))
#add faltten layer
model.add(Flatten())

In [10]:
#add hidden layers
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [12]:
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 158s 11s/step - loss: 4.3152 - accuracy: 0.5138 - val_loss: 1.3933 - val_accuracy: 0.5950
Epoch 2/10
14/14 [==============================] - 27s 2s/step - loss: 0.6349 - accuracy: 0.7546 - val_loss: 0.2791 - val_accuracy: 0.9339
Epoch 3/10
14/14 [==============================] - 27s 2s/step - loss: 0.3146 - accuracy: 0.8509 - val_loss: 0.1317 - val_accuracy: 0.9669
Epoch 4/10
14/14 [==============================] - 27s 2s/step - loss: 0.2458 - accuracy: 0.8830 - val_loss: 0.1175 - val_accuracy: 0.9669
Epoch 5/10
14/14 [==============================] - 27s 2s/step - loss: 0.2096 - accuracy: 0.9037 - val_loss: 0.0908 - val_accuracy: 0.9669
Epoch 6/10
14/14 [==============================] - 28s 2s/step - loss: 0.1872 - accuracy: 0.9266 - val_loss: 0.1333 - val_accuracy: 0.9339
Epoch 7/10
14/14 [==============================] - 27s 2s/step - loss: 0.1830 - accuracy: 0.9266 - val_loss: 0.0821 - val_accuracy: 0.9752
Epoch 8/10
14/14 [

In [13]:
model.save("forest.h5")

In [10]:
!tar -zcvf ImageAnalysis_new.tgz forest.h5

tar: forest.h5: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [11]:
ls  -1

Dataset/
forest1.h5
Image-Analysis_new.tgz
ImageAnalysis_new.tgz
ImageAnalysis.tgz


In [12]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 32.9 MB/s eta 0:00:01


In [13]:
from ibm_watson_machine_learning import APIClient
wml_credentilas = {
                   "url":"https://us-south.ml.cloud.ibm.com",
                   "apikey":"jl9RoTgGKtspv-nJJ5fA1ztkvrSUJsrbiAidgi7GKqCR"
}
client = APIClient(wml_credentilas)

In [14]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])


In [18]:
space_uid = guid_from_space_name(client, 'imageclassification')
print("Space UID = " + space_uid)

Space UID = 7d664c97-e46a-4b77-b0f9-8817cc1d4517


In [19]:
client.set.default_space(space_uid)

'SUCCESS'

In [20]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [21]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [22]:
model_details = client.repository.store_model(model='ImageAnalysis_new.tgz',meta_props={ 
client.repository.ModelMetaNames.NAME:'CNN',
client.repository.ModelMetaNames.TYPE:"tensorflow_rt22.1",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                             )
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [23]:
model_id

'c9086b18-7ef1-494d-b05c-52b8b65071ed'

In [ ]:
pred=model.predict(x)

In [16]:
pred

array([[0.]], dtype=float32)

# Open cv for video processing


In [26]:
pip install twilio

     |████████████████████████████████| 1.4 MB 27.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install playsound

  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7037 sha256=82b7de9bf65aa10f5bc9ce92ac16f134b8c6f36456293610de62d4bf876770ca
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ba/39/54/c8f7ff9a88a644d3c58b4dec802d90b79a2e0fb2a6b884bf82
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.


In [29]:
from logging import WARNING
#import opencv library
import cv2
#import numpy
import numpy as np
#import image function from keras
from keras.preprocessing import image
#import load_model from keras
from keras.models import load_model
#import client from twilio API
from twilio.rest import Client
#import playsound package
from playsound import playsound

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


# Creating An Account in Twilio Service

In [24]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='klNf6oE0FdirMnVg_5lZlC9luxa3pINITIX48MlCWJu5',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'imageanalysis-donotdelete-pr-3zp79iioz5e4oo'
object_key = 'Video_analysis.ipynb'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
from twilio.rest import Client
from playsound import playsound
if pred==0:
  print('Forest fire')
  account_sid='AC2a45c94082d15fd26a52ef4274bc95bd'
  auth_token='769fa5108da21ef101aba8ef9ee8ce73'
  client=Client(account_sid,auth_token)
  message=client.messages \
  .create(
      body='forest fire is detected,stay alert',
      #use twilio free number
      from_='+18316035766',
      #to number
      to='+916383605324')
  print(message.sid)
  print("Fire detected")
  print("SMS Sent!")
elif pred==1:
  print('No Danger')